# Load packages

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Load data

In [2]:
df = px.data.stocks(indexed=False)
df

,date,GOOG,AAPL,AMZN,FB,NFLX,MSFT
0,2018-01-01,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,2018-01-08,1.018172,1.011943,1.061881,0.959968,1.053526,1.015988
2,2018-01-15,1.032008,1.019771,1.053240,0.970243,1.049860,1.020524
3,2018-01-22,1.066783,0.980057,1.140676,1.016858,1.307681,1.066561
4,2018-01-29,1.008773,0.917143,1.163374,1.018357,1.273537,1.040708
...,...,...,...,...,...,...,...
100,2019-12-02,1.216280,1.546914,1.425061,1.075997,1.463641,1.720717
101,2019-12-09,1.222821,1.572286,1.432660,1.038855,1.421496,1.752239
102,2019-12-16,1.224418,1.596800,1.453455,1.104094,1.604362,1.784896
103,2019-12-23,1.226504,1.656000,1.521226,1.113728,1.567170,1.802472


# Wide to Long table

In [3]:
id_vars = ["date"]
value_vars = [col for col in df.columns if col not in id_vars]
df_long = pd.melt(
    df, id_vars=id_vars, value_vars=value_vars, 
    var_name="ID", value_name="Value",
    ignore_index=True
)
df_long["date"] = pd.to_datetime(df_long["date"])
df_long

,date,ID,Value
0,2018-01-01,GOOG,1.000000
1,2018-01-08,GOOG,1.018172
2,2018-01-15,GOOG,1.032008
3,2018-01-22,GOOG,1.066783
4,2018-01-29,GOOG,1.008773
...,...,...,...
625,2019-12-02,MSFT,1.720717
626,2019-12-09,MSFT,1.752239
627,2019-12-16,MSFT,1.784896
628,2019-12-23,MSFT,1.802472


# Plot multiple time series colored by ID

In [4]:
fig = px.line(
    df_long, x="date", y="Value", color="ID",
    hover_data={"date": "|%B %d, %Y"},
    title='custom tick labels'
)

# update x-axis
fig.update_xaxes(
    dtick="M1",            # distance between two ticks
    tickformat="%b\n%Y",
    ticklabelmode="period" # Moving Tick Labels to the Middle of the Period
)

# Alternative, update layout
# fig.update_layout(
#     xaxis_dtick="M1",
#     xaxis_tickformat = '%d %B (%a)<br>%Y',
#     xaxis_ticklabelmode="period"
# )
fig.show()

# Plot using graph object

In [5]:
col_id = "ID"
col_time = "date"
col_x = "Value"

fig = go.Figure() # Create a figure

# Iterately add a trace
for ID, sub_df in df_long.groupby(by=col_id):
    fig.add_trace(
        # add graph_objects use go
        go.Scatter(
            x=sub_df[col_time],
            y=sub_df[col_x],
            mode='lines',
            name=f'{col_id} {ID}',
            hovertext=sub_df[col_time].dt.strftime('%B %d, %Y')  # Format the hover text
        )
    )

# update x-axis
fig.update_xaxes(
    dtick="M1",            # distance between two ticks
    tickformat="%b\n%Y",
    ticklabelmode="period"  # Moving Tick Labels to the Middle of the Period
)

# update title
fig.update_layout(
    title='Custom Tick Labels'
)

fig.show()

# Compute simple return

$$
R_t = \frac{P_t - P_{t-1}}{P_{t-1}} = \frac{P_t}{P_{t-1}} - 1
$$

In [6]:
df_long['R_t'] = df_long.groupby(by=col_id, as_index=False)[col_x]\
    .transform(lambda x: (x/x.shift(1) - 1))
df_long

,date,ID,Value,R_t
0,2018-01-01,GOOG,1.000000,NaN
1,2018-01-08,GOOG,1.018172,0.018172
2,2018-01-15,GOOG,1.032008,0.013589
3,2018-01-22,GOOG,1.066783,0.033696
4,2018-01-29,GOOG,1.008773,-0.054378
...,...,...,...,...
625,2019-12-02,MSFT,1.720717,0.002444
626,2019-12-09,MSFT,1.752239,0.018320
627,2019-12-16,MSFT,1.784896,0.018637
628,2019-12-23,MSFT,1.802472,0.009847


# Dual axis plot with grouped legend

Can only be created using graph objects.

In [7]:
def hex_to_rgb(hex_color: str) -> str:
    """change hex color code to rgb color code

    Example
    ---------
    >>> hex_color = '#FFAABB'
    >>> hex_to_rgb(hex_color)
    >>> (255, 170, 187)
    """
    hex_color = hex_color.lstrip('#')  # Remove the '#' character if it's present
    return tuple(int(hex_color[i:i + 2], 16) for i in (0, 2, 4))

In [8]:
color_palette = px.colors.qualitative.Alphabet

fig = make_subplots(specs=[[{"secondary_y": True}]])

# Iterately add a trace
for i, (ID, sub_df) in enumerate(df_long.groupby(by=col_id)):
    # note some of color code are hex, some are rgb
    color = str(hex_to_rgb(color_palette[i]))
    color_P = f"rgba{color[:-1]}, 1)"      # opacity/alpha = 1
    color_R = f"rgba{color[:-1]}, 0.7)"    # opacity/alpha = 0.7

    fig.add_trace(go.Scatter(
        x=sub_df[col_time],
        y=sub_df[col_x],
        mode='lines',
        name='P_t',
        legendgroup=ID,                    # Specify the legend group
        legendgrouptitle_text=f"ID: {ID}", # Specify the legend group title
        line=dict(color=color_P, width=2),
        yaxis='y'  # Specify the primary y-axis
    ))

    fig.add_trace(go.Scatter(
        x=sub_df[col_time],
        y=sub_df['R_t'],
        mode='lines',
        name=f'R_t',
        legendgroup=ID,                    # Specify the legend group
        line=dict(color=color_R, width=2),
        yaxis='y2'  # Specify the secondary y-axis
    ))

color = str(hex_to_rgb(color_palette[i+5]))
color_P = f"rgba{color[:-1]}, 1)"      
color_R = f"rgba{color[:-1]}, 0.7)"    

fig.update_layout(
    title=dict(
        text="<b>Stocks P_t and R_t<b>", # <b> to set bold
        font=dict(
            size=20,
            family="Times New Roman",
            color="blue",
        ),
    ),
    yaxis=dict(
        title="P_t",
        titlefont=dict(
            color=color_P
        ),
        tickfont=dict(
            color=color_P
        ),
        anchor="x",
    ),
    yaxis2=dict(
        title="R_t",
        titlefont=dict(
            color=color_R
        ),
        tickfont=dict(
            color=color_R
        ),
        tickmode="sync", # sync the axis
        anchor="x",
        overlaying="y",  # overlap with y
        side="right",
    ),
)

fig.show()


# Colors

## Qualitative

In [9]:
import plotly.express as px

fig = px.colors.qualitative.swatches()
fig.show()

print(px.colors.qualitative.Plotly)

['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52']


## Sequential

In [10]:
import plotly.express as px

fig = px.colors.sequential.swatches_continuous()
fig.show()

print(px.colors.sequential.Viridis)

['#440154', '#482878', '#3e4989', '#31688e', '#26828e', '#1f9e89', '#35b779', '#6ece58', '#b5de2b', '#fde725']


## Diverging

In [11]:
import plotly.express as px

fig = px.colors.diverging.swatches_continuous()
fig.show()

# Reference

- [Time Series and Date Axes in Python](https://plotly.com/python/time-series/)
- [Multiple Axes in Python](https://plotly.com/python/multiple-axes/)
- [Discrete Colors in Python](https://plotly.com/python/discrete-color/)
- [Built-in Continuous Color Scales in Python](https://plotly.com/python/builtin-colorscales/)